# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [5]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [6]:
# Your code here
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [7]:
# Your code here
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)
train_err = mean_squared_error(y_train, y_hat_train)
test_err = mean_squared_error(y_test, y_hat_test)
print('Training MSE: ', train_err)
print('Test MSE: ', test_err)

Training MSE:  0.16900862815073664
Test MSE:  0.135504063703792


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [52]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    # add 1 to fold size to account for leftovers  
    
    df_split = np.array_split(data, k)

    return df_split

### Apply it to the Ames Housing data

In [108]:
# Make sure to concatenate the data again
ames_data = pd.concat([ames_log_norm, ames_ohe], axis=1)

In [54]:
# Apply kfolds() to ames_data with 5 folds
ames_split = kfolds(ames_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [129]:
train_errs = []
test_errs = []

k=5

for n in range(k):
    # Split in train and test for the fold
    test = ames_split[n]
    train = ames_data.drop(test.index)
    
    # Fit a linear regression model
    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']
    X_test = test.drop('SalePrice_log', axis=1)
    y_test = test['SalePrice_log']
    
    linreg.fit(X_train, y_train)
    
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_err = mean_squared_error(y_train, y_hat_train)
    test_err = mean_squared_error(y_test, y_hat_test)
    
    train_errs.append(train_err)
    test_errs.append(test_err)

train_errs_mean = np.mean(train_errs)  
test_errs_mean = np.mean(test_errs)  

print('Train errs mean: ', train_errs_mean)
print('Test errs mean: ', test_errs_mean)

Train errs mean:  0.15927599576611778
Test errs mean:  0.17702834210001123


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [146]:
# Your code here
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

cv_5_results = cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error')
cv_5_results.mean()

-0.17702834210001123

In [147]:
# From solution branch

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
cv_5_results.mean()

0.17702834210001123

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 